<a href="https://colab.research.google.com/github/pabloperfer/CUDAexercises/blob/main/stridegridloop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from math import hypot
from numba import cuda

@cuda.jit
def hypot_stride(a, b, c):
    idx = cuda.grid(1)
    stride = cuda.gridsize(1)

    for x in range(idx, a.shape[0], stride):
        c[x] = hypot(a[x], b[x])

In [4]:
import numpy as np
from numba import cuda
n = 1000000
a = np.random.uniform(-12, 12, n).astype(np.float32)
b = np.random.uniform(-12, 12, n).astype(np.float32)
d_a = cuda.to_device(a)
d_b = cuda.to_device(b)
d_c = cuda.device_array_like(d_b)

blocks = 128
threads_per_block = 64

hypot_stride[blocks, threads_per_block](d_a, d_b, d_c)

In [5]:
from numpy import testing
testing.assert_almost_equal(np.hypot(a, b), d_c.copy_to_host(), decimal=5)